In [1]:
from pyspark.sql import SparkSession
from pandas import set_option

set_option('display.max.columns', None)

spark = (
    SparkSession
    .builder
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

spark.conf.set('spark.sql.adaptive.enabled', True)
spark.conf.set('spark.sql.session.timeZone', 'UTC')

### Silver

In [2]:
import os

PATH = '/tmp/requests'
BRONZE = os.path.join(PATH, 'bronze')
SILVER = os.path.join(PATH, 'silver')

### Employees

In [27]:
%%time
path = os.path.join(SILVER, 'employees')

(
    spark
    .read
    .parquet(os.path.join(BRONZE, 'Employees'))
    .where(col('FullName') != 'OnBase Processing Service')
    .selectExpr(
        'ID as EMPLOYEE_ID',
        'FullName as EMPLOYEE_NAME',
        'PrimaryRoleName as ROLE',
        'lower(Email) as EMAIL',
        'ManagerName as MANAGER_NAME',
        'PodName as TEAM'
    )
    .write
    .mode('overwrite')
    .parquet(path)
)

employees = spark.read.parquet(path)
employees.limit(5).toPandas()

CPU times: user 18.3 ms, sys: 3.68 ms, total: 21.9 ms
Wall time: 853 ms


,EMPLOYEE_ID,EMPLOYEE_NAME,ROLE,EMAIL,MANAGER_NAME,TEAM
0,194728,Andi Prins,Audit Supervisor,aprins@spendmend.com,Travis Wheeler,Gold
1,194729,Bob VanGoor,Audit Supervisor,bvangoor@spendmend.com,Dan Hutchins,Red
2,194730,Colleen Kretowicz,Audit Supervisor,ckretowicz@spendmend.com,Travis Wheeler,Gold
3,194731,Cindy Allen,WNC Auditor,callen@spendmend.com,None,None
4,194732,Dan Hutchins,Audit Manager,dhutchins@spendmend.com,Dan Hutchins,Red


### Requests

In [56]:
%%time
from pyspark.sql.functions import col, lit, lower

column = when(lower('Contact').contains('@'), lower('Contact'))

path = os.path.join(SILVER, 'requests')


(
    spark
    .read
    .parquet(os.path.join(BRONZE, 'StatementRequests'))
    .withColumn('Contact', column)
    .withColumnRenamed('RequesterFullName', 'EMPLOYEE_NAME')
    .join(
        employees
            .select('EMPLOYEE_ID', 'EMPLOYEE_NAME'),
        on='EMPLOYEE_NAME',
        how='left'
    )
    .selectExpr(
        'ObjectID as REQUEST_ID',
        'ReferenceNumber as REFERENCE_ID',
        'EMPLOYEE_ID',
        'JobNo as JOB_ID',
        'ReferenceNumber as REFERENCE_NUMBER',
        'cast(RequestDate as date) as REQUEST_DATE',
        'Status as REQUEST_STATUS',
        'RequestMethod as REQUEST_METHOD',
        'RequestType as REQUEST_TYPE',
        'cast(LastActivityDate as date) as LAST_ACTIVITY_DATE',
        'cast(LastStatementReceivedDate as date) as LAST_RECEIVED_DATE',
        'CustomerName as CUSTOMER_NAME',
        'VendorNo as VENDOR_NUMBER',
        'cast(WNC as boolean) as WILL_NOT_COMPLY',
        'cast(StatementWNC as boolean) as STATEMENT_WILL_NOT_COMPLY',
        'cast(WNCSpecialHandling as boolean) as HAS_SPECIAL_HANDLING',
        'VendorGroupName as VENDOR_GROUP_NAME',
        'cast(Volume as double) as VOLUME',
        'VolumeTier as VOLUME_LEVEL',
        'cast(VolumeLast12 as double) as VOLUME_PREVIOUS_YEAR',
        'CallsheetNo as CALL_SHEET_NUMBER',
        'CallerStatus as CALL_STATUS',
        'cast(WebsiteVendor as boolean) as VENDOR_HAS_WEBSITE',
        'cast(NeedLeadVendor as boolean) as NEEDS_LEAD_VENDOR'
        
    )
    .write
    .mode('overwrite')
    .parquet(path)
)


requests = spark.read.parquet(path)
# requests.createOrReplaceTempView('requests')
# display(requests)
requests.limit(5).toPandas()

CPU times: user 37.9 ms, sys: 5.41 ms, total: 43.3 ms
Wall time: 6.27 s


,REQUEST_ID,REFERENCE_ID,EMPLOYEE_ID,JOB_ID,REFERENCE_NUMBER,REQUEST_DATE,REQUEST_STATUS,REQUEST_METHOD,REQUEST_TYPE,LAST_ACTIVITY_DATE,LAST_RECEIVED_DATE,CUSTOMER_NAME,VENDOR_NUMBER,WILL_NOT_COMPLY,STATEMENT_WILL_NOT_COMPLY,HAS_SPECIAL_HANDLING,VENDOR_GROUP_NAME,VOLUME,VOLUME_LEVEL,VOLUME_PREVIOUS_YEAR,CALL_SHEET_NUMBER,CALL_STATUS,VENDOR_HAS_WEBSITE,NEEDS_LEAD_VENDOR
0,47597380,NaN,NaN,NaN,NaN,None,New,None,Caller,None,None,None,None,None,None,None,None,NaN,None,NaN,None,None,None,None
1,26681846,1285660.0,8398642.0,3427.0,1285660.0,2020-07-24,Superceded,MassEmail,None,2020-07-24,None,New Hanover Regional Medical Center,16845,None,None,None,A M COFFEE DISTRIBUTORS LLC,0.0,B,0.0,C-338914,None,None,None
2,26819776,1285660.0,8398642.0,3427.0,1285660.0,2020-07-31,Superceded,MassEmail,Mass 1,2020-07-31,None,New Hanover Regional Medical Center,16845,None,None,None,A M COFFEE DISTRIBUTORS LLC,0.0,B,0.0,C-338914,None,None,None
3,30690711,1285660.0,8398642.0,3427.0,1285660.0,2021-01-19,Superceded,MassEmail,Caller,2020-07-31,None,New Hanover Regional Medical Center,16845,None,None,None,A M COFFEE DISTRIBUTORS LLC,0.0,B,0.0,C-338914,Sent Authorization Letter,None,None
4,36365526,1285660.0,194731.0,3427.0,1285660.0,2021-08-19,Superceded,MassEmail,Mass 1,2021-08-23,None,New Hanover Regional Medical Center,16845,None,None,None,A M COFFEE DISTRIBUTORS LLC,0.0,B,0.0,C-338914,Sent Authorization Letter,None,None


### Activities

In [99]:
%%time
from pyspark.sql import Window
from pyspark.sql.functions import col, count, date_trunc, desc, dense_rank, isnull, length, lower, when

conditions = [
    col('NOTES').contains('contact information updated'),
    col('NOTES').contains('statement request in project tracker'),
    isnull('NOTES'),
    isnull('REFERENCE_ID')
]

condition = ~(conditions[0] | conditions[1] | conditions[2] | conditions[3])
window = dense_rank().over(Window.partitionBy(condition, 'REFERENCE_ID').orderBy(desc('ACTIVITY_DATE')))
noteRank = when(condition, window)

# activityColumn = count('NOTE_REFERENCE_RANK').over(window)


condition = ~(isnull('EMPLOYEE_ID') | isnull('REFERENCE_ID'))
partitions = [
    condition,
    col('EMPLOYEE_ID'),
    col('REFERENCE_ID')
]
window = dense_rank().over(Window.partitionBy(*partitions).orderBy(desc('ACTIVITY_DATE')))
employeeRank = when(condition, window)


contactColumn = when(col('ContactType') != 'N/A', col('ContactType'))
noteColumn = when(length('Notes') < 4096, col('Notes'))

mapping = create_map(
    lit('Called Vendor'), lit('CALL'),
    lit('Note Only'), lit('NOTE'),
    lit('Emailed Vendor'), lit('EMAIL'),
    lit('Received Call / Email'), lit('RESPONSE')
)


path = os.path.join(SILVER, 'activities')

(
    spark
    .read
    .parquet(os.path.join(BRONZE, 'StatementRequestActivityRecords'))
    .withColumn('ActivityType', mapping[col('ActivityType')])
    .withColumn('ContactType', contactColumn)
    .withColumn('Notes', noteColumn)
    .withColumnRenamed('ActivityUser', 'EMPLOYEE_NAME')
    .join(
        employees
            .select('EMPLOYEE_ID', 'EMPLOYEE_NAME'),
        on='EMPLOYEE_NAME',
        how='left'
    )
    .selectExpr(
        'ObjectID as ACTIVITY_ID',
        'ReferenceNumber as REFERENCE_ID',
        'VendorContactObjectID as CONTACT_ID',
        'StatementRequestObjectID as REQUEST_ID',
        'EMPLOYEE_ID',
        'CreatedDate as ACTIVITY_DATE',
        'ActivityType as ACTIVITY',
        'JobNumber as JOB_NUMBER',
        'JobName as JOB_NAME',
        'CustomerVendorName as VENDOR_NAME',
        'cast(ReferenceNumber as string) as REFERENCE_NUMBER',
        'Outcome as OUTCOME',
        'Notes as NOTES'
    )
    .withColumn('EMPLOYEE_REFERENCE_RANK', employeeRank)
    .withColumn('NOTE_REFERENCE_RANK', noteRank)
    .withColumn('ACTIVITY_DATE', date_trunc('second', 'ACTIVITY_DATE'))
    .write
    .mode('overwrite')
    .parquet(path)
)


activities = spark.read.parquet(path)
# activities.createOrReplaceTempView('activities')
# display(activities)
activities.limit(5).toPandas()

CPU times: user 56.1 ms, sys: 10.8 ms, total: 66.9 ms
Wall time: 17.7 s


,ACTIVITY_ID,REFERENCE_ID,CONTACT_ID,REQUEST_ID,EMPLOYEE_ID,ACTIVITY_DATE,ACTIVITY,JOB_NUMBER,JOB_NAME,VENDOR_NAME,REFERENCE_NUMBER,OUTCOME,NOTES,EMPLOYEE_REFERENCE_RANK,NOTE_REFERENCE_RANK
0,31330781,1048712,NaN,NaN,20493525.0,2021-02-12 21:03:10,None,NaN,None,None,1048712,None,None,1.0,NaN
1,31065034,1048712,NaN,31065028.0,20493525.0,2021-01-29 12:02:52,None,2775.0,BJC Healthcare - 2775,DCM GROUP,1048712,Sent Authorization Letter,None,2.0,NaN
2,29833679,1048712,NaN,NaN,20420366.0,2020-12-21 11:47:44,None,NaN,None,None,1048712,None,None,1.0,NaN
3,29491514,1048712,NaN,28751407.0,NaN,2020-12-09 09:43:14,None,2775.0,BJC Healthcare - 2775,DCM GROUP,1048712,None,None,NaN,NaN
4,47767498,1048720,NaN,NaN,NaN,2022-05-09 08:50:49,None,NaN,None,None,1048720,None,The Statement Preferred Vendor Contact in OnBa...,NaN,NaN


### Statements

In [206]:
%%time
path = os.path.join(SILVER, 'statements')

column = dense_rank().over(Window.partitionBy('REFERENCE_ID').orderBy(desc('STATEMENT_DATE')))

( 
    spark
    .read
    .parquet(os.path.join(BRONZE, 'Statements'))
    .selectExpr(
        'ReferenceNumber as REFERENCE_ID',
        'abs(xxhash64(EmailMessageID)) as EMAIL_ID',
        'CreatedDate as STATEMENT_DATE',
    )
    .withColumn('REFERENCE_RANK', column)
    .withColumn('STATEMENT_DATE', date_trunc('second', 'STATEMENT_DATE'))
#     .join(
#         activities,
#         on='REFERENCE_ID',
#         how='left_semi'
#     )
    .write
    .mode('overwrite')
    .parquet(path)
)


statements = spark.read.parquet(path)
# statements.createOrReplaceTempView('statements')
# display(statements)
statements.limit(5).toPandas()

CPU times: user 21.6 ms, sys: 4.27 ms, total: 25.9 ms
Wall time: 1.65 s


,REFERENCE_ID,EMAIL_ID,STATEMENT_DATE,REFERENCE_RANK
0,1048708,7565227526545052117,2022-05-26 08:55:42,1
1,1048708,4285445397631831328,2022-02-18 15:52:41,2
2,1048708,3553086900216677347,2021-11-16 13:39:07,3
3,1048708,7649443478599343877,2021-08-13 16:29:35,4
4,1048708,1098207253644678064,2021-06-23 08:53:09,5


### Processes
- workitemlc
- rmobject

In [26]:
%%time
from pyspark.sql import Window
from pyspark.sql.functions import date_trunc, dense_rank, desc

window = Window.partitionBy('contentnum').orderBy(desc('transdate'))

path = os.path.join(SILVER, 'processes')

# primary key (contentnum, lcnum)
(
    spark.read.parquet(os.path.join(BRONZE, 'WorkItems'))
    .withColumn('transdate', date_trunc('second', 'transdate'))
    .select('contentnum', 'lcnum', 'statenum', 'transdate')
    .dropDuplicates()
    .withColumn('_dense_rank', dense_rank().over(window))
    .join(
        spark.read.parquet(os.path.join(BRONZE, 'Objects'))
        .selectExpr('objectid as contentnum', 'activestatus'),
        on='contentnum',
        how='inner'
    )
    .selectExpr(
        'contentnum as PROCESS_ID',
        'lcnum as PROCESS_CODE',
        'statenum as STATE_CODE',
        'activestatus as STATUS_CODE',
        'transdate as TRANSACTION_DATE',
        '_dense_rank as PROCESS_RANK'
    )
    .write
    .mode('overwrite')
    .parquet(path)
)

processes = spark.read.parquet(path)
processes.limit(5).toPandas()

CPU times: user 31.9 ms, sys: 7.65 ms, total: 39.6 ms
Wall time: 27.8 s


,PROCESS_ID,PROCESS_CODE,STATE_CODE,STATUS_CODE,TRANSACTION_DATE,PROCESS_RANK
0,194780,126,208,0,2018-08-06 09:40:25,1
1,194784,126,208,0,2016-07-26 00:29:57,1
2,291093,134,252,0,2021-05-08 01:35:53,1
3,315261,126,208,0,2016-07-21 18:29:16,1
4,404196,124,222,1,2016-07-19 14:49:20,1


### Jobs

In [15]:
%%time
path = os.path.join(SILVER, 'jobs')

(
    spark.read.parquet(os.path.join(BRONZE, 'Jobs'))
    .selectExpr(
        'JobNo as JOB_ID',
        'ManagerID as MANAGER_ID',
        'SupervisorID as SUPERVISOR_ID',
        'JobNo as JOB_NUMBER',
        'ManagerPodName as TEAM'
    )
    .write
    .mode('overwrite')
    .parquet(path)
)

jobs = spark.read.parquet(path)
jobs.limit(5).toPandas()

CPU times: user 16 ms, sys: 3.44 ms, total: 19.4 ms
Wall time: 532 ms


,JOB_ID,MANAGER_ID,SUPERVISOR_ID,JOB_NUMBER,TEAM
0,2600,NaN,NaN,2600,None
1,3570,NaN,NaN,3570,None
2,3178,NaN,NaN,3178,None
3,3447,NaN,NaN,3447,None
4,3467,NaN,NaN,3467,None


### States
- lcstate

In [23]:
%%time
path = os.path.join(SILVER, 'states')

(
    spark.read.parquet(os.path.join(BRONZE, 'States'))
    .selectExpr(
        'statenum as STATE_CODE',
        'statename as STATE_NAME'
    )
    .write
    .mode('overwrite')
    .parquet(path)
)

states = spark.read.parquet(path)
states.limit(5).toPandas()

CPU times: user 13.2 ms, sys: 3.1 ms, total: 16.3 ms
Wall time: 434 ms


,STATE_CODE,STATE_NAME
0,185,SYS - Initial
1,186,SYS - Done
2,187,Claim Image Errors
3,188,SYS - Initial
4,189,Assignment


### Notes

In [59]:
%%time
from pyspark.sql import Window
from pyspark.sql.functions import col, date_trunc, desc, dense_rank, length, lower, when

column = when(length('STNDescription') < 4096, col('STNDescription'))

condition = lower('STNDescription').contains('statement re-released')
window = dense_rank().over(Window.partitionBy(condition, 'STID').orderBy(desc('STNAdded')))
case = when(condition, window)


path = os.path.join(SILVER, 'descriptions')

(
    spark.read.parquet(os.path.join(BRONZE, 'Notes'))
    .join(
        requests,
        on=requests['REFERENCE_ID'] == notes['STID'],
        how='left_semi'
    )
    .withColumn('_length', column)
    .withColumn('_row_number', case)
    .withColumn('STNAdded', date_trunc('second', 'STNAdded'))
    .selectExpr(
        'STID as REFERENCE_ID',
        'STNAdded as DESCRIPTION_DATE',
        '_length as STATEMENT_DESCRIPTION',
        '_row_number as RELEASE_RANK'
    )
    .write
    .mode('overwrite')
    .parquet(path)
)


descriptions = spark.read.parquet(path)
descriptions.limit(5).toPandas()

CPU times: user 30.9 ms, sys: 25.7 ms, total: 56.7 ms
Wall time: 34 s


,REFERENCE_ID,DESCRIPTION_DATE,STATEMENT_DESCRIPTION,RELEASE_RANK
0,1048708,2021-02-10,None,NaN
1,1048722,2021-01-22,None,NaN
2,1048722,2020-12-24,None,NaN
3,1048725,2021-02-16,None,NaN
4,1048732,2021-03-23,None,NaN


### Projects

In [63]:
%%time
from pyspark.sql import Window
from pyspark.sql.functions import col, create_map, dense_rank, desc, lit

mapping = create_map(
    lit('Closed'), lit('Closed'),
    lit('Open'), lit('Open'),
    lit('Prep'), lit('Prepared'),
    lit('Pull'), lit('Pull'),
    lit('Review'), lit('Review')
)


# appears Closed usually has an EndDate
# name is JobNo + ProjectType
column = dense_rank().over(Window.partitionBy('JobNo', 'ProjectType').orderBy('StartDate'))

path = os.path.join(SILVER, 'projects')

(
    spark.read.parquet(os.path.join(BRONZE, 'Projects'))
    .withColumn('Status', mapping[col('Status')])
    .withColumn('_dense_rank', column)
    .selectExpr(
        'JobNo as JOB_ID',
        'ProjectType as PROJECT_TYPE',
        'Status as STATUS',
        'cast(StartDate as date) as START_DATE',
        'cast(EndDate as date) as END_DATE',
        '_dense_rank as PROJECT_RANK'
    )
    .write
    .mode('overwrite')
    .parquet(path)
)

projects = spark.read.parquet(path)
projects.limit(5).toPandas()

CPU times: user 29.4 ms, sys: 6.18 ms, total: 35.5 ms
Wall time: 1.04 s


,JOB_ID,PROJECT_TYPE,STATUS,START_DATE,END_DATE,PROJECT_RANK
0,2489,Returns,Open,NaT,NaT,1
1,2509,Dupes,Open,NaT,NaT,1
2,2617,Returns,Open,NaT,NaT,1
3,2659,Dupes,Open,NaT,NaT,1
4,2761,AP vs PO,Closed,NaT,2019-09-04,1


### Claims

In [113]:
%%time
path = os.path.join(SILVER, 'claims')

(
    spark
    .read
    .parquet(os.path.join(BRONZE, 'Claims'))
    .where('JID IS NOT NULL')
    .selectExpr(
        'CLAIMID as CLAIM_ID',
        'JID as JOB_ID',
        'CLVendorCode as VENDOR_NUMBER',
        'CLStatus as STATUS',
        'cast(CLAmount as double) as AMOUNT'
    )
    .write
    .mode('overwrite')
    .parquet(path)
)

claims = spark.read.parquet(path)
claims.limit(5).toPandas()

CPU times: user 19.2 ms, sys: 4.59 ms, total: 23.8 ms
Wall time: 1.38 s


,CLAIM_ID,JOB_ID,VENDOR_NUMBER,STATUS,AMOUNT
0,126365,1282,8405,2,3979.00
1,128887,1282,003039,2,6017.00
2,129310,1282,002134,5,190.56
3,129311,1282,002134,5,1148.00
4,129312,1282,002134,5,124.67


### Report

In [207]:
%%time
names = os.listdir(SILVER)
names.sort()

for name in names:
    path = os.path.join(SILVER, name)
    spark.read.parquet(path).createOrReplaceTempView(name)

CPU times: user 13.5 ms, sys: 4.75 ms, total: 18.2 ms
Wall time: 1.06 s


In [230]:
# file = os.path.join(
#     os.path.expanduser('~'),
#     'Desktop',
#     'SpendMend',
#     'projects',
#     'call-reports',
#     'sql',
#     'RequestsSpark.sql'
# )

with open(file, mode='r') as stream:
    query = stream.read()

In [231]:
(
    spark
    .sql(query)
    # .count()
    .limit(5)
    .toPandas()
)

,STATE_NAME,TEAM,MANAGER_NAME,SUPERVISOR_NAME,REQUEST_ID,REFERENCE_ID,EMPLOYEE_ID,JOB_ID,REFERENCE_NUMBER,REQUEST_DATE,REQUEST_STATUS,REQUEST_METHOD,REQUEST_TYPE,LAST_ACTIVITY_DATE,LAST_RECEIVED_DATE,CUSTOMER_NAME,VENDOR_NUMBER,WILL_NOT_COMPLY,STATEMENT_WILL_NOT_COMPLY,HAS_SPECIAL_HANDLING,VENDOR_GROUP_NAME,VOLUME,VOLUME_LEVEL,VOLUME_PREVIOUS_YEAR,CALL_SHEET_NUMBER,CALL_STATUS,VENDOR_HAS_WEBSITE,NEEDS_LEAD_VENDOR,STATEMENT_DATE,ACTIVITY_DAYS,ACTIVITY_DATE,NOTES,ACTIVITY_COUNT,EMPLOYEE_NAME,VENDOR_CLAIMS,VENDOR_CLAIMS_AMOUNT
0,Supervisor,Gold,Travis Wheeler,Travis Wheeler,37277893,1048715,194731,2775,1048715,2021-09-10,No Receipt,AdHoc,Caller,2022-04-15,None,BJC Healthcare,0000012376,None,True,None,BIOCOMPATIBLES INC,2661770.00,AA,379625.00,C-344044,Needs Supervisor,None,None,2020-08-18 12:10:47,94,2022-03-18,emailed Barbara.Schaarschmidt@bsci.com for all...,82,Alyssa Neiser,6171,20169449.32
1,Statement Calling,Gold,Travis Wheeler,Travis Wheeler,37885110,1048721,8404677,2775,1048721,2021-10-01,No Receipt,WNC,Caller,2022-06-03,None,BJC Healthcare,0000001623,None,True,None,GLOBUS MEDICAL,2097656.00,AA,2868477.42,C-312523,Will Not Comply,None,None,2021-05-06 16:27:54,17,2022-06-03,"No response, resent to ar@globusmedical.com",65,Alyssa Neiser,6171,20169449.32
2,Statement Calling,Gold,Travis Wheeler,Travis Wheeler,36855776,1048801,194731,2775,1048801,2021-09-03,No Receipt,MassFax,Caller,2021-08-23,None,BJC Healthcare,0000004058,False,False,None,MERCURY DISTRIBUTING,27886.75,B,35172.10,C-411873,Sent Authorization Letter,False,False,NaT,290,2021-09-03,Statement Request sent successfully from Mass/...,7,None,6171,20169449.32
3,Awaiting Account Identification,Gold,Travis Wheeler,Travis Wheeler,43235250,1048812,194731,2775,1048812,2022-03-07,Account Identification,AdHoc,Caller,2022-03-24,2022-03-24,BJC Healthcare,0000000233,False,None,None,NUCLEAR DIAGNOSIS INC,264127.95,A,261428.98,C-305074,Statement Received,None,None,2022-03-24 16:11:22,88,2022-03-24,Statement received in the office. Status chang...,33,None,6171,20169449.32
4,Statement Calling,Gold,Travis Wheeler,Travis Wheeler,48232374,1048818,12611472,2775,1048818,2022-05-27,No Receipt,WNC,Caller,2022-06-15,None,BJC Healthcare,0000005828,False,True,None,FORTEC MEDICAL INC,142506.00,A,173144.00,C-308446,Will Not Comply,None,None,2022-02-25 11:19:56,5,2022-06-15,1st request remote Accounts.Receivable@forte...,38,Becky Williams,6171,20169449.32


In [225]:
query = """
SELECT
    datediff(current_date(), activities.ACTIVITY_DATE) as ACTIVITY_DAYS,
    cast(activities.ACTIVITY_DATE as date) as ACTIVITY_DATE,
    activities.NOTES,
    count(ACTIVITY_ID) OVER (
        PARTITION BY
            REFERENCE_ID
    ) as ACTIVITY_COUNT,
    NOTE_REFERENCE_RANK
FROM
    activities
WHERE
    NOTE_REFERENCE_RANK IS NOT NULL
"""


(
    spark
    .sql(query)
    .limit(5)
    .toPandas()
)

,ACTIVITY_DAYS,ACTIVITY_DATE,NOTES,ACTIVITY_COUNT,NOTE_REFERENCE_RANK
0,15,2022-06-05,"per Travis, low volume",7,1
1,290,2021-09-03,Statement Request sent successfully from Mass/...,7,2
2,290,2021-09-03,Email Initiated from the MassEmail System in BSAP,7,3
3,301,2021-08-23,Statement Request sent successfully from Mass/...,7,4
4,302,2021-08-22,Email Initiated from the MassEmail System in BSAP,7,5


In [210]:
spark.read.table('statements')

DataFrame[REFERENCE_ID: int, EMAIL_ID: bigint, STATEMENT_DATE: timestamp, REFERENCE_RANK: int]

In [ ]:
spark.stop()